In [1]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold,RandomizedSearchCV,cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
from sklearn.model_selection import KFold, RepeatedKFold


In [2]:
data = pd.read_excel('train_final.xlsx')

In [3]:
#data = data[data['project_is_approved'].isnull()!=True]
data.head()

,school_state,project_grade_category,project_subject_categories,teacher_number_of_previously_posted_projects,project_is_approved,items,quantity,price,total_price,min_quantity,...,resource_text_need,resource_text_reading,resource_text_school,resource_text_students,resource_text_use,resource_text_work,gender,Project_SubCategory_1,Project_SubCategory_2,Project_SubCategory_3
0,33,3,24,26,1,2,6,299.98,899.94,3,...,0.165534,0.208328,0.346046,0.801055,0.177267,0.170913,0,16,-1,-1
1,10,0,42,1,0,1,20,20.00,400.00,20,...,0.000000,0.000000,0.803034,0.557678,0.000000,0.000000,0,23,26,-1
2,44,0,36,5,1,1,1,469.99,469.99,1,...,0.091018,0.000000,0.190271,0.605624,0.000000,0.000000,0,0,17,-1
3,27,0,8,16,0,5,5,684.47,684.47,1,...,0.181486,0.000000,0.695556,0.658688,0.000000,0.000000,1,14,-1,-1
4,4,1,8,42,1,1,2,355.50,711.00,2,...,0.000000,0.000000,0.000000,0.948588,0.000000,0.000000,1,14,-1,-1


In [4]:
y = data['project_is_approved']
X = data[['teacher_number_of_previously_posted_projects','items', 'quantity',
              'price','min_quantity', 'min_price',
              'mean_price','mean_total_price','essay2_len','essay3_len','word_count',
              'word_density','title_word_count','upper_case_word_count',
              'article_text_classroom','article_text_learning','article_text_reading','article_text_students',
              'article_text_use','resource_text_classroom','resource_text_reading','resource_text_students',
              'gender','Project_SubCategory_1']]
#X = data.drop(['teacher_prefix','text','project_is_approved','project_submitted_datetime','teacher_id','id','project_essay_1','project_essay_2','project_essay_3','project_essay_4','project_title','project_resource_summary'],axis=1)

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42, stratify = y)

#lasso = linear_model.Lasso(alpha = 4.5e-5, normalize=True)

#lasso.fit(X,y)

#lasso_coef = lasso.fit(X,y).coef_
#lasso_index = np.where(lasso_coef!=0)
#print(lasso_coef)

In [10]:
cat_score

array([0.73013173, 0.72270703, 0.72624607, 0.72547687])

In [25]:
# 5. Catboosting
estimator = CatBoostClassifier(iterations=100)
param_cat = {'depth' : range(3,10),
             'learning_rate': [0.03, 0.1],
             'l2_leaf_reg': [1, 3, 5, 7, 9]}


inner_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

cat = RandomizedSearchCV(estimator, param_cat, cv=inner_cv)
cat.fit(X,y)
cat_score = cross_val_score(cat, X=X, y=y, cv=outer_cv,scoring='roc_auc')

cat_score

0:	learn: 0.6722295	total: 53.1ms	remaining: 5.26s
1:	learn: 0.6533866	total: 89.4ms	remaining: 4.38s
2:	learn: 0.6354234	total: 122ms	remaining: 3.94s
3:	learn: 0.6188774	total: 151ms	remaining: 3.63s
4:	learn: 0.6039572	total: 178ms	remaining: 3.37s
5:	learn: 0.5897312	total: 205ms	remaining: 3.21s
6:	learn: 0.5768409	total: 230ms	remaining: 3.06s
7:	learn: 0.5644536	total: 254ms	remaining: 2.92s
8:	learn: 0.5526654	total: 290ms	remaining: 2.93s
9:	learn: 0.5420127	total: 323ms	remaining: 2.91s
10:	learn: 0.5320697	total: 346ms	remaining: 2.8s
11:	learn: 0.5225844	total: 366ms	remaining: 2.69s
12:	learn: 0.5136312	total: 389ms	remaining: 2.6s
13:	learn: 0.5057459	total: 419ms	remaining: 2.57s
14:	learn: 0.4982042	total: 444ms	remaining: 2.51s
15:	learn: 0.4919648	total: 477ms	remaining: 2.5s
16:	learn: 0.4854229	total: 508ms	remaining: 2.48s
17:	learn: 0.4792138	total: 531ms	remaining: 2.42s
18:	learn: 0.4737376	total: 554ms	remaining: 2.36s
19:	learn: 0.4684403	total: 575ms	remainin

65:	learn: 0.3904716	total: 2s	remaining: 1.03s
66:	learn: 0.3900738	total: 2.03s	remaining: 1s
67:	learn: 0.3897289	total: 2.06s	remaining: 968ms
68:	learn: 0.3894248	total: 2.08s	remaining: 936ms
69:	learn: 0.3890756	total: 2.11s	remaining: 905ms
70:	learn: 0.3887253	total: 2.13s	remaining: 872ms
71:	learn: 0.3884367	total: 2.16s	remaining: 840ms
72:	learn: 0.3881559	total: 2.19s	remaining: 809ms
73:	learn: 0.3878768	total: 2.21s	remaining: 777ms
74:	learn: 0.3876370	total: 2.23s	remaining: 744ms
75:	learn: 0.3873641	total: 2.26s	remaining: 713ms
76:	learn: 0.3870982	total: 2.28s	remaining: 681ms
77:	learn: 0.3868363	total: 2.3s	remaining: 650ms
78:	learn: 0.3866459	total: 2.33s	remaining: 619ms
79:	learn: 0.3864044	total: 2.35s	remaining: 588ms
80:	learn: 0.3861860	total: 2.38s	remaining: 558ms
81:	learn: 0.3859822	total: 2.4s	remaining: 527ms
82:	learn: 0.3857807	total: 2.43s	remaining: 497ms
83:	learn: 0.3856209	total: 2.45s	remaining: 466ms
84:	learn: 0.3854348	total: 2.48s	remai

32:	learn: 0.4227090	total: 891ms	remaining: 1.81s
33:	learn: 0.4206555	total: 916ms	remaining: 1.78s
34:	learn: 0.4185595	total: 938ms	remaining: 1.74s
35:	learn: 0.4167737	total: 959ms	remaining: 1.71s
36:	learn: 0.4150812	total: 985ms	remaining: 1.68s
37:	learn: 0.4134573	total: 1.01s	remaining: 1.65s
38:	learn: 0.4120347	total: 1.03s	remaining: 1.61s
39:	learn: 0.4106879	total: 1.05s	remaining: 1.58s
40:	learn: 0.4092320	total: 1.08s	remaining: 1.55s
41:	learn: 0.4080032	total: 1.1s	remaining: 1.52s
42:	learn: 0.4067194	total: 1.12s	remaining: 1.49s
43:	learn: 0.4056223	total: 1.14s	remaining: 1.45s
44:	learn: 0.4046022	total: 1.16s	remaining: 1.42s
45:	learn: 0.4036120	total: 1.18s	remaining: 1.39s
46:	learn: 0.4025869	total: 1.21s	remaining: 1.36s
47:	learn: 0.4015552	total: 1.23s	remaining: 1.33s
48:	learn: 0.4005921	total: 1.25s	remaining: 1.3s
49:	learn: 0.3998670	total: 1.28s	remaining: 1.28s
50:	learn: 0.3991058	total: 1.3s	remaining: 1.25s
51:	learn: 0.3983406	total: 1.32s	

98:	learn: 0.3762455	total: 2.12s	remaining: 21.4ms
99:	learn: 0.3761014	total: 2.15s	remaining: 0us
0:	learn: 0.6266402	total: 40.7ms	remaining: 4.02s
1:	learn: 0.5783784	total: 65.5ms	remaining: 3.21s
2:	learn: 0.5383721	total: 89.4ms	remaining: 2.89s
3:	learn: 0.5079533	total: 110ms	remaining: 2.63s
4:	learn: 0.4834486	total: 131ms	remaining: 2.48s
5:	learn: 0.4638925	total: 151ms	remaining: 2.37s
6:	learn: 0.4491433	total: 170ms	remaining: 2.25s
7:	learn: 0.4371078	total: 188ms	remaining: 2.17s
8:	learn: 0.4282740	total: 207ms	remaining: 2.1s
9:	learn: 0.4209871	total: 227ms	remaining: 2.04s
10:	learn: 0.4145201	total: 246ms	remaining: 1.99s
11:	learn: 0.4095102	total: 266ms	remaining: 1.95s
12:	learn: 0.4054994	total: 284ms	remaining: 1.9s
13:	learn: 0.4019589	total: 304ms	remaining: 1.87s
14:	learn: 0.3995008	total: 323ms	remaining: 1.83s
15:	learn: 0.3972459	total: 342ms	remaining: 1.79s
16:	learn: 0.3953025	total: 361ms	remaining: 1.76s
17:	learn: 0.3936761	total: 382ms	remaini

64:	learn: 0.3788006	total: 1.3s	remaining: 699ms
65:	learn: 0.3787178	total: 1.31s	remaining: 677ms
66:	learn: 0.3786495	total: 1.33s	remaining: 656ms
67:	learn: 0.3785484	total: 1.35s	remaining: 635ms
68:	learn: 0.3784497	total: 1.37s	remaining: 615ms
69:	learn: 0.3782936	total: 1.39s	remaining: 595ms
70:	learn: 0.3781735	total: 1.41s	remaining: 574ms
71:	learn: 0.3780474	total: 1.42s	remaining: 554ms
72:	learn: 0.3779671	total: 1.44s	remaining: 533ms
73:	learn: 0.3777666	total: 1.46s	remaining: 514ms
74:	learn: 0.3776102	total: 1.48s	remaining: 494ms
75:	learn: 0.3775570	total: 1.5s	remaining: 474ms
76:	learn: 0.3774461	total: 1.52s	remaining: 454ms
77:	learn: 0.3774005	total: 1.54s	remaining: 433ms
78:	learn: 0.3773358	total: 1.55s	remaining: 413ms
79:	learn: 0.3772399	total: 1.57s	remaining: 393ms
80:	learn: 0.3771514	total: 1.59s	remaining: 373ms
81:	learn: 0.3770951	total: 1.61s	remaining: 353ms
82:	learn: 0.3770297	total: 1.63s	remaining: 333ms
83:	learn: 0.3769545	total: 1.65s

...

31:	learn: 0.4331892	total: 464ms	remaining: 986ms
32:	learn: 0.4308660	total: 486ms	remaining: 988ms
33:	learn: 0.4286093	total: 511ms	remaining: 991ms
34:	learn: 0.4264454	total: 531ms	remaining: 986ms
35:	learn: 0.4247624	total: 553ms	remaining: 983ms
36:	learn: 0.4229303	total: 569ms	remaining: 970ms
37:	learn: 0.4211823	total: 588ms	remaining: 960ms
38:	learn: 0.4195810	total: 612ms	remaining: 958ms
39:	learn: 0.4183527	total: 628ms	remaining: 943ms
40:	learn: 0.4168268	total: 648ms	remaining: 933ms
41:	learn: 0.4154557	total: 665ms	remaining: 918ms
42:	learn: 0.4142036	total: 680ms	remaining: 901ms
43:	learn: 0.4130199	total: 696ms	remaining: 886ms
44:	learn: 0.4120023	total: 714ms	remaining: 872ms
45:	learn: 0.4108608	total: 730ms	remaining: 857ms
46:	learn: 0.4098311	total: 748ms	remaining: 843ms
47:	learn: 0.4089725	total: 764ms	remaining: 827ms
48:	learn: 0.4082133	total: 780ms	remaining: 812ms
49:	learn: 0.4074367	total: 797ms	remaining: 797ms
50:	learn: 0.4066128	total: 814

93:	learn: 0.3925281	total: 1.47s	remaining: 94.1ms
94:	learn: 0.3923780	total: 1.49s	remaining: 78.4ms
95:	learn: 0.3921956	total: 1.51s	remaining: 62.8ms
96:	learn: 0.3921050	total: 1.52s	remaining: 47.2ms
97:	learn: 0.3919790	total: 1.54s	remaining: 31.5ms
98:	learn: 0.3918745	total: 1.56s	remaining: 15.8ms
99:	learn: 0.3917286	total: 1.59s	remaining: 0us
0:	learn: 0.6736115	total: 17.8ms	remaining: 1.76s
1:	learn: 0.6552479	total: 33.2ms	remaining: 1.63s
2:	learn: 0.6380052	total: 47.9ms	remaining: 1.55s
3:	learn: 0.6221890	total: 60ms	remaining: 1.44s
4:	learn: 0.6073005	total: 70.8ms	remaining: 1.34s
5:	learn: 0.5933691	total: 82ms	remaining: 1.28s
6:	learn: 0.5803742	total: 93.2ms	remaining: 1.24s
7:	learn: 0.5682487	total: 105ms	remaining: 1.2s
8:	learn: 0.5574100	total: 118ms	remaining: 1.19s
9:	learn: 0.5469934	total: 128ms	remaining: 1.16s
10:	learn: 0.5372234	total: 139ms	remaining: 1.13s
11:	learn: 0.5279332	total: 150ms	remaining: 1.1s
12:	learn: 0.5196851	total: 161ms	re

68:	learn: 0.3977244	total: 834ms	remaining: 375ms
69:	learn: 0.3973418	total: 846ms	remaining: 363ms
70:	learn: 0.3970940	total: 855ms	remaining: 349ms
71:	learn: 0.3967813	total: 867ms	remaining: 337ms
72:	learn: 0.3964893	total: 877ms	remaining: 324ms
73:	learn: 0.3961590	total: 888ms	remaining: 312ms
74:	learn: 0.3959483	total: 899ms	remaining: 300ms
75:	learn: 0.3957087	total: 912ms	remaining: 288ms
76:	learn: 0.3954719	total: 923ms	remaining: 276ms
77:	learn: 0.3952876	total: 934ms	remaining: 264ms
78:	learn: 0.3950582	total: 944ms	remaining: 251ms
79:	learn: 0.3948165	total: 954ms	remaining: 239ms
80:	learn: 0.3946150	total: 965ms	remaining: 226ms
81:	learn: 0.3943736	total: 976ms	remaining: 214ms
82:	learn: 0.3941923	total: 987ms	remaining: 202ms
83:	learn: 0.3940268	total: 997ms	remaining: 190ms
84:	learn: 0.3938542	total: 1.01s	remaining: 178ms
85:	learn: 0.3936490	total: 1.02s	remaining: 166ms
86:	learn: 0.3934804	total: 1.04s	remaining: 155ms
87:	learn: 0.3933487	total: 1.0

35:	learn: 0.3792644	total: 1.17s	remaining: 2.09s
36:	learn: 0.3790245	total: 1.21s	remaining: 2.05s
37:	learn: 0.3787286	total: 1.24s	remaining: 2.02s
38:	learn: 0.3784972	total: 1.27s	remaining: 1.98s
39:	learn: 0.3780936	total: 1.3s	remaining: 1.95s
40:	learn: 0.3778934	total: 1.33s	remaining: 1.91s
41:	learn: 0.3776032	total: 1.36s	remaining: 1.88s
42:	learn: 0.3772831	total: 1.4s	remaining: 1.85s
43:	learn: 0.3770829	total: 1.43s	remaining: 1.81s
44:	learn: 0.3767995	total: 1.46s	remaining: 1.78s
45:	learn: 0.3765809	total: 1.49s	remaining: 1.75s
46:	learn: 0.3763277	total: 1.52s	remaining: 1.71s
47:	learn: 0.3761087	total: 1.55s	remaining: 1.68s
48:	learn: 0.3759035	total: 1.58s	remaining: 1.65s
49:	learn: 0.3757582	total: 1.61s	remaining: 1.61s
50:	learn: 0.3753819	total: 1.64s	remaining: 1.58s
51:	learn: 0.3751373	total: 1.68s	remaining: 1.55s
52:	learn: 0.3749505	total: 1.71s	remaining: 1.51s
53:	learn: 0.3747759	total: 1.74s	remaining: 1.48s
54:	learn: 0.3745799	total: 1.77s

array([0.72833098, 0.72359375, 0.72637814, 0.72685061])